# Requirement 2: Data Reconciliation to Trial Balance

---

## ANSWER TO REQUIREMENT

**Requirement 2a:** The $84,867,855 sales revenue is comprised of **1,158 sales order records** with InvoiceDate in calendar year 2017.

**Requirement 2b:** The $11,988,886 gross accounts receivable is comprised of **148 invoice records** that were outstanding (unpaid) as of 12/31/2017.

### Key Findings:
- **Revenue Recognition:** 1,158 of 1,168 total sales orders (99.1%) were invoiced in 2017
- **AR Composition:** The 148 outstanding invoices include:
  - 31 invoices still unpaid as of data extraction (PaidDate = 9/9/9999)
  - 117 invoices paid AFTER 12/31/2017 (but were in AR at year-end)
- **Collection Performance:** 87.2% of 2017 invoices were collected by year-end (1,010 paid by 12/31/2017)
- **Reconciliation:** Both amounts match the trial balance exactly ($0.08 rounding difference on AR)

### Recommended Actions:
- Perform aging analysis on the 148 AR invoices to identify collection risk (Requirement 6)
- Investigate the 31 invoices still unpaid - some may be significantly past due
- Review Q4 spike in revenue and its impact on year-end AR balance
- Analyze payment patterns - why were 117 invoices paid in early 2018?

---

## Data Sources
- `UMD_Data Set_Sales Orders.xlsx`
- `UMD_Data Set_Customer Invoices.xlsx`

## Outputs Generated
- `outputs/tables/revenue_records_2017.csv` - Full list of 1,158 revenue transactions
- `outputs/tables/ar_outstanding_12_31_2017.csv` - Full list of 148 AR invoices as of year-end
- `outputs/tables/reconciliation_summary.csv` - Summary reconciliation to trial balance


## Setup & Data Loading


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Create output directories if they don't exist
Path('../outputs/tables').mkdir(parents=True, exist_ok=True)
Path('../outputs/figures').mkdir(parents=True, exist_ok=True)

print("✓ Libraries loaded and output directories ready")


✓ Libraries loaded and output directories ready


In [ ]:
# Load data files
data_dir = Path('../data')

sales_orders = pd.read_excel(data_dir / 'UMD_Data Set_Sales Orders.xlsx')
invoices = pd.read_excel(data_dir / 'UMD_Data Set_Customer Invoices.xlsx')

print(f"✓ Sales Orders loaded: {len(sales_orders):,} records")
print(f"✓ Customer Invoices loaded: {len(invoices):,} records")


✓ Sales Orders loaded: 1,168 records
✓ Customer Invoices loaded: 1,167 records


## Requirement 2a: Identify Sales Revenue Records ($84,867,855)

**Key Concept:** Per the case, "The Company invoices customers when the goods are shipped, and invoicing triggers the recording of revenue." Therefore, revenue is recognized when InvoiceDate occurs.

**Logic:** 
- Revenue is recorded when invoices are issued (not when orders are placed)
- We need all sales orders with InvoiceDate in calendar year 2017
- The SubTotal field contains the amount included in Sales Revenue (per data dictionary)


In [ ]:
# Join sales orders with invoices to get InvoiceDate
# This links each sale to when it was actually invoiced (revenue recognition date)
revenue_analysis = sales_orders.merge(
    invoices[['InvoiceID', 'InvoiceDate']], 
    on='InvoiceID', 
    how='left'
)

# Filter for invoices dated in 2017
revenue_analysis['InvoiceYear'] = revenue_analysis['InvoiceDate'].dt.year
revenue_2017 = revenue_analysis[revenue_analysis['InvoiceYear'] == 2017].copy()

# Calculate total revenue
total_revenue = revenue_2017['SubTotal'].sum()
record_count = len(revenue_2017)
trial_balance_revenue = 84_867_855

print("=" * 80)
print("REVENUE RECONCILIATION - Requirement 2a")
print("=" * 80)
print(f"\nTrial Balance - Sales Revenue: ${trial_balance_revenue:,.2f}")
print(f"Calculated Revenue (2017 invoices): ${total_revenue:,.2f}")
print(f"Difference: ${total_revenue - trial_balance_revenue:,.2f}")
print(f"\nNumber of sales order records: {record_count:,}")
print(f"\nPercentage of total orders: {record_count / len(sales_orders) * 100:.1f}%")
print("=" * 80)


REVENUE RECONCILIATION - Requirement 2a

Trial Balance - Sales Revenue: $84,867,855.00
Calculated Revenue (2017 invoices): $84,867,855.00
Difference: $0.00

Number of sales order records: 1,158

Percentage of total orders: 99.1%


### Revenue Distribution by Quarter

Let's examine when the 2017 revenue was recognized to understand the timing pattern (particularly Q4 spike mentioned in the case).


In [ ]:
# Analyze revenue by quarter
revenue_2017['Quarter'] = revenue_2017['InvoiceDate'].dt.quarter
quarterly_revenue = revenue_2017.groupby('Quarter').agg({
    'SubTotal': 'sum',
    'SalesOrderID': 'count'
}).rename(columns={'SalesOrderID': 'Record_Count'})

quarterly_revenue['Percent_of_Revenue'] = (quarterly_revenue['SubTotal'] / total_revenue * 100)

print("\nQuarterly Revenue Breakdown for 2017:")
print("-" * 60)
for quarter in [1, 2, 3, 4]:
    if quarter in quarterly_revenue.index:
        rev = quarterly_revenue.loc[quarter, 'SubTotal']
        count = quarterly_revenue.loc[quarter, 'Record_Count']
        pct = quarterly_revenue.loc[quarter, 'Percent_of_Revenue']
        print(f"Q{quarter}: ${rev:>15,.2f} ({count:>4,.0f} records, {pct:>5.1f}%)")
print("-" * 60)



Quarterly Revenue Breakdown for 2017:
------------------------------------------------------------
Q1: $  19,268,120.00 ( 257 records,  22.7%)
Q2: $  21,248,380.00 ( 286 records,  25.0%)
Q3: $  20,104,965.00 ( 282 records,  23.7%)
Q4: $  24,246,390.00 ( 333 records,  28.6%)
------------------------------------------------------------


## Requirement 2b: Identify Accounts Receivable Records ($11,988,886)

**Key Concept:** AR as of 12/31/2017 represents all invoices that were OUTSTANDING (not yet paid) as of that date. This includes invoices that were later paid in 2018.

**Logic:**
- Find all 2017 invoices where PaidDate is AFTER 12/31/2017 (includes paid in 2018 + unpaid marked 9/9/9999)
- An invoice dated in 2017 but paid in 2018 was still in AR on 12/31/2017
- Join to SalesOrders to get the TotalDue amount (AR balance = revenue + tax + freight)


In [ ]:
# Identify AR as of 12/31/2017
# AR includes invoices that had not been paid as of the balance sheet date

# Get all 2017 invoices
invoices_2017 = invoices[invoices['InvoiceDate'].dt.year == 2017].copy()

# Convert PaidDate to datetime for comparison
invoices_2017['PaidDate_dt'] = pd.to_datetime(invoices_2017['PaidDate'], errors='coerce')

# AR = invoices where PaidDate is AFTER 12/31/2017 (or never paid)
# This includes:
# 1. Invoices paid in 2018 or later (PaidDate > 12/31/2017)
# 2. Invoices still unpaid (PaidDate = 9/9/9999, which becomes 9999-09-09)
ar_invoices = invoices_2017[
    (invoices_2017['PaidDate_dt'] > '2017-12-31') |
    (invoices_2017['PaidDate'].astype(str).str.contains('9999'))
].copy()

# Join to sales orders to get TotalDue amount
ar_detail = ar_invoices.merge(
    sales_orders[['InvoiceID', 'SalesOrderID', 'TotalDue', 'SubTotal', 'CustID', 'TerritoryID']], 
    on='InvoiceID', 
    how='left'
)

# Calculate total AR
total_ar = ar_detail['TotalDue'].sum()
ar_record_count = len(ar_detail)
trial_balance_ar = 11_988_886

# Breakdown by payment status
still_unpaid = ar_invoices[ar_invoices['PaidDate'].astype(str).str.contains('9999')]
paid_later = ar_invoices[ar_invoices['PaidDate_dt'] > '2017-12-31']

print("=" * 80)
print("ACCOUNTS RECEIVABLE RECONCILIATION - Requirement 2b")
print("=" * 80)
print(f"\nTrial Balance - Gross AR: ${trial_balance_ar:,.2f}")
print(f"Calculated AR (outstanding as of 12/31/2017): ${total_ar:,.2f}")
print(f"Difference: ${total_ar - trial_balance_ar:,.2f}")
print(f"\nNumber of outstanding invoice records: {ar_record_count:,}")
print(f"  - Still unpaid (as of data extraction): {len(still_unpaid):,}")
print(f"  - Paid after 12/31/2017: {len(paid_later):,}")
print(f"\nPercentage of 2017 invoices in AR at year-end: {ar_record_count / len(revenue_2017) * 100:.1f}%")
print("=" * 80)


ACCOUNTS RECEIVABLE RECONCILIATION - Requirement 2b

Trial Balance - Gross AR: $11,988,886.00
Calculated AR (outstanding as of 12/31/2017): $11,988,886.08
Difference: $0.08

Number of outstanding invoice records: 148
  - Still unpaid (as of data extraction): 31
  - Paid after 12/31/2017: 117

Percentage of 2017 invoices in AR at year-end: 12.8%


### AR Distribution Analysis

Let's examine when these AR invoices were issued and when they were paid to understand collection patterns.


In [ ]:
# Analyze AR by invoice quarter to see if Q4 spike created AR concentration
ar_detail['InvoiceQuarter'] = ar_detail['InvoiceDate'].dt.to_period('Q')
quarterly_ar = ar_detail.groupby('InvoiceQuarter')['TotalDue'].agg(['sum', 'count'])
quarterly_ar.columns = ['TotalDue', 'Invoice_Count']

print("\nAR by Invoice Quarter:")
print("-" * 70)
print(f"{'Quarter':<12} {'AR Balance':>15} {'Count':>8} {'% of Total AR':>12}")
print("-" * 70)
for quarter in quarterly_ar.index:
    balance = quarterly_ar.loc[quarter, 'TotalDue']
    count = quarterly_ar.loc[quarter, 'Invoice_Count']
    pct = (balance / total_ar * 100)
    print(f"{str(quarter):<12} ${balance:>14,.2f} {count:>8.0f} {pct:>11.1f}%")
print("-" * 70)

# Analyze payment timing for those paid after year-end
ar_paid_later = ar_detail[ar_detail['PaidDate_dt'] <= '2018-12-31'].copy()  # Exclude 9999 dates
if len(ar_paid_later) > 0:
    ar_paid_later['PaymentMonth'] = ar_paid_later['PaidDate_dt'].dt.to_period('M')
    payment_timing = ar_paid_later.groupby('PaymentMonth')['TotalDue'].agg(['count', 'sum'])
    payment_timing.columns = ['Count', 'Amount']
    payment_timing = payment_timing.head(10)
    
    print("\nPayment Timing for Invoices Paid After 12/31/2017:")
    print("-" * 60)
    print(f"{'Month':<12} {'Count':>8} {'Amount':>15}")
    print("-" * 60)
    for month in payment_timing.index:
        count = payment_timing.loc[month, 'Count']
        amount = payment_timing.loc[month, 'Amount']
        print(f"{str(month):<12} {count:>8.0f} ${amount:>14,.2f}")
    print("-" * 60)

# Summary statistics
print(f"\nAR Statistics:")
print(f"  Average invoice: ${ar_detail['TotalDue'].mean():,.2f}")
print(f"  Median invoice: ${ar_detail['TotalDue'].median():,.2f}")
print(f"  Largest invoice: ${ar_detail['TotalDue'].max():,.2f}")
print(f"  Smallest invoice: ${ar_detail['TotalDue'].min():,.2f}")



AR by Invoice Quarter:
----------------------------------------------------------------------
Quarter           AR Balance    Count % of Total AR
----------------------------------------------------------------------
2017Q3       $    317,935.92        5         2.7%
2017Q4       $ 11,670,950.16      143        97.3%
----------------------------------------------------------------------

Payment Timing for Invoices Paid After 12/31/2017:
------------------------------------------------------------
Month           Count          Amount
------------------------------------------------------------
2018-01           117 $  9,318,713.62
------------------------------------------------------------

AR Statistics:
  Average invoice: $81,005.99
  Median invoice: $72,377.46
  Largest invoice: $262,878.00
  Smallest invoice: $11,889.44


In [ ]:
# Save revenue records for documentation
revenue_export = revenue_2017[[
    'SalesOrderID', 'InvoiceID', 'InvoiceDate', 'CustID', 'TerritoryID',
    'ProdID', 'Quantity', 'UnitPrice', 'SubTotal', 'TotalDue'
]].sort_values('InvoiceDate')

revenue_export.to_csv('../outputs/tables/revenue_records_2017.csv', index=False)
print(f"✓ Saved {len(revenue_export):,} revenue records to outputs/tables/revenue_records_2017.csv")

# Save AR records for documentation
# Use the already-merged ar_detail dataframe and select available columns
available_cols = list(ar_detail.columns)
export_col_map = {}

# Build the export selecting the correct column names
for col in ['InvoiceID', 'InvoiceDate', 'SalesOrderID', 'CustID', 'TerritoryID', 'SubTotal', 'TotalDue', 'PaidDate']:
    if col in available_cols:
        export_col_map[col] = col
    elif f'{col}_x' in available_cols:
        export_col_map[col] = f'{col}_x'
    elif f'{col}_y' in available_cols:
        export_col_map[col] = f'{col}_y'

# Select and rename columns
ar_export = ar_detail[[export_col_map[col] for col in export_col_map.keys()]].copy()
ar_export.columns = list(export_col_map.keys())

# Sort by TotalDue descending (largest balances first)
ar_export = ar_export.sort_values('TotalDue', ascending=False)

# Add a flag to show which are still unpaid vs paid later
ar_export['Status'] = ar_export['PaidDate'].apply(
    lambda x: 'Unpaid' if '9999' in str(x) else 'Paid in 2018'
)

ar_export.to_csv('../outputs/tables/ar_outstanding_12_31_2017.csv', index=False)
print(f"✓ Saved {len(ar_export):,} AR records to outputs/tables/ar_outstanding_12_31_2017.csv")

# Create reconciliation summary
reconciliation_summary = pd.DataFrame({
    'Account': ['Sales Revenue', 'Accounts Receivable'],
    'Trial_Balance': [trial_balance_revenue, trial_balance_ar],
    'Calculated_Total': [total_revenue, total_ar],
    'Difference': [total_revenue - trial_balance_revenue, total_ar - trial_balance_ar],
    'Record_Count': [record_count, ar_record_count],
    'Match': ['Perfect', 'Perfect (within $0.08)']
})

reconciliation_summary.to_csv('../outputs/tables/reconciliation_summary.csv', index=False)
print(f"✓ Saved reconciliation summary to outputs/tables/reconciliation_summary.csv")

print("\n" + "=" * 80)
print("All output files saved successfully")
print("=" * 80)


✓ Saved 1,158 revenue records to outputs/tables/revenue_records_2017.csv
✓ Saved 148 AR records to outputs/tables/ar_outstanding_12_31_2017.csv
✓ Saved reconciliation summary to outputs/tables/reconciliation_summary.csv

All output files saved successfully


---

## Summary & Audit Implications

### What the Data Shows

Our reconciliation successfully tied the trial balance amounts to the underlying transaction data with **near-perfect accuracy**:

1. **Revenue ($84,867,855):** Comprises 1,158 sales orders that were invoiced during calendar year 2017. This represents 99.1% of all sales orders in the database, indicating that nearly all transactions were completed and invoiced in 2017.

2. **Accounts Receivable ($11,988,886):** Comprises 148 invoices that were outstanding (unpaid) as of 12/31/2017. These break down as:
   - 31 invoices still unpaid as of data extraction (21%)
   - 117 invoices paid in early 2018 (79%)
   - The 148 invoices represent 12.8% of 2017 invoices, meaning 87.2% were collected by year-end

### Why This Matters for the Audit

**Strong Collection Performance - Or Is It?** At first glance, an 87% collection rate within the same year appears healthy. However, several factors raise concerns:
- The AR balance of $12M represents 14.1% of annual revenue, which is relatively high
- 117 invoices were paid in early 2018 - this could indicate normal 30-60 day terms OR collection issues
- 31 invoices remain unpaid - need to determine how old these are (aging analysis)

**Q4 Revenue Spike Impact:** The case mentions Q4 2017 sales increased 214% YoY. Key questions:
- Are the 148 AR invoices concentrated in Q4? (analyzed in this notebook)
- Were Q4 sales legitimate or the result of channel stuffing / bill-and-hold arrangements?
- Did distributors request extended payment terms to take Q4 shipments?

**Bill-and-Hold Concerns:** The case mentions "the Company agreed to hold the inventory in their own warehouse" for some distributors. This raises serious revenue recognition issues:
- If goods weren't delivered, revenue recognition may be premature
- Need to identify which of the 148 AR invoices (if any) relate to bill-and-hold arrangements
- Such arrangements would violate proper revenue recognition principles

**Affected Assertions:**
- **Existence/Occurrence:** Need to verify the 148 AR invoices represent legitimate, delivered sales
- **Cutoff:** Critical to ensure Q4 revenue was recognized in the correct period (not bill-and-hold)
- **Valuation:** Need to assess collectability of the $12M AR, especially the 31 still-unpaid invoices
- **Accuracy:** Verify amounts are calculated correctly

### What Further Testing is Recommended

1. **Aging Analysis (Requirement 6):** Determine how many of the 148 AR invoices are >90 days past due as of 12/31/2017
2. **Bill-and-Hold Investigation:** Identify which AR invoices (if any) involve inventory held at UMD's warehouse
3. **Three-Way Match (Requirement 3):** Confirm all 1,158 revenue transactions have proper documentation (order → shipment → invoice)
4. **Credit Limit Testing (Requirements 4-5):** Verify if any AR balances exceed authorized credit limits
5. **Q4 Revenue Testing:** Examine shipping documents for Q4 sales to verify actual delivery to customers
6. **Customer Concentration:** Identify if the $12M AR is concentrated with specific customers or territories (potential fraud indicator)
7. **Subsequent Collections:** Verify that the 117 invoices marked as "paid in 2018" were actually collected
